In [3]:
from sqlite3 import connect
import pandas as pd

conn = connect("machine2.db")

df = pd.read_sql_query("SELECT participantId, instanceId, agreement_level from personality", conn)
df

,participantId,instanceId,agreement_level
0,97f4_7385706431,8,None
1,97f4_7385706431,25,None
2,97f4_7385706431,28,None
3,97f4_7385706431,30,None
4,97f4_7385706431,9,None
...,...,...,...
47156,zzru_7377077714,32,4
47157,zzru_7377077714,11,2
47158,zzru_7377077714,2,4
47159,zzru_7377077714,33,4


In [8]:
mappings = {
  'extraversion': [1, 6, 11, 16, 21, 26, 31, 36],
  'agreeableness': [2, 7, 12, 17, 22, 27, 32, 37, 42],
  'conscientiousness': [3, 8, 13, 18, 23, 28, 33, 38, 43],
  'neuroticism': [4, 9, 14, 19, 24, 29, 34, 39],
  'openness':[5, 10, 15, 20, 25, 30, 35, 40, 41, 44]
}
n_questions = 44

inverse_mappings = [None] * (n_questions + 1)
for category, questions in mappings.items():
  for question in questions:
    inverse_mappings[question] = category

revs = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]

valid = [1, 2, 3, 4, 5]


In [12]:
valid_df = df[df['agreement_level'].isin(valid)].copy()
valid_df.loc[valid_df['instanceId'].isin(revs), 'agreement_level'] = 6 - valid_df.loc[valid_df['instanceId'].isin(revs), 'agreement_level']
valid_df['group'] = valid_df['instanceId'].map(lambda x: inverse_mappings[x])
result = valid_df.groupby(['participantId', 'group'])['agreement_level'].mean().reset_index()
result.head(n=10)

,participantId,group,agreement_level
0,a265_7382527621,agreeableness,4.222222
1,a265_7382527621,conscientiousness,2.666667
2,a265_7382527621,extraversion,2.125
3,a265_7382527621,neuroticism,4.875
4,a265_7382527621,openness,2.9
5,a295_7382607228,agreeableness,3.666667
6,a295_7382607228,conscientiousness,2.666667
7,a295_7382607228,extraversion,1.625
8,a295_7382607228,neuroticism,3.25
9,a295_7382607228,openness,3.0


In [ ]:
def check_complete(series: pd.Series):
  mask = series.isin(valid)
  n_values = mask.sum()
  if n_values == len(series):
    return 1
  if n_values == 0:
    return -1
  return 0


agg = result.pivot_table(index='participantId', columns='group', values='agreement_level')

complete = df.groupby('participantId')['agreement_level'].apply(check_complete)
agg['complete'] = agg.index.map(complete.to_dict())
agg['mean'] = agg.index.map(valid_df.groupby('participantId')['agreement_level'].mean().to_dict())

agg.columns.name = None
agg.reset_index(inplace=True)
agg.head(n = 10)

,participantId,agreeableness,conscientiousness,extraversion,neuroticism,openness,complete,mean
0,a265_7382527621,4.222222,2.666667,2.125,4.875,2.9,1,3.340909
1,a295_7382607228,3.666667,2.666667,1.625,3.25,3.0,1,2.863636
2,a3a7_7375337601,3.777778,4.0,4.5,3.125,4.2,1,3.931818
3,a3kp_7384516585,2.666667,2.333333,3.75,4.5,4.8,1,3.613636
4,a3rz_7384305362,4.222222,3.666667,4.25,3.571429,3.4,1,3.813953
5,a3wk_7382757539,4.0,4.111111,3.25,2.75,4.3,1,3.727273
6,a4hb_7384347504,2.888889,4.0,3.75,3.125,3.4,1,3.431818
7,a53i_7374966509,4.333333,2.777778,2.375,3.125,4.5,1,3.477273
8,a5e4_7384928308,4.0,2.777778,3.625,3.875,4.0,1,3.659091
9,a73q_7384417134,2.0,3.0,3.125,4.5,4.3,1,3.386364


In [18]:
agg.to_csv('personality.csv')